In [21]:
!python3 -m pip install transformers
# !pip3 show transformers

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForTokenClassification, AdamW

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/21/02/ae8e595f45b6c8edee07913892b3b41f5f5f273962ad98851dc6a564bbb9/transformers-4.31.0-py3-none-any.whl.metadata
  Using cached transformers-4.31.0-py3-none-any.whl.metadata (116 kB)
Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\Dell\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\transformers\\models\\deprecated\\trajectory_transformer\\convert_trajectory_transformer_original_pytorch_checkpoint_to_pytorch.py'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



ModuleNotFoundError: No module named 'transformers.utils'

In [ ]:
data = [
    {"text": "2022 óleo sobre tela", "label": "óleo sobre tela"},
    {"text": "Obra faz parte da série Enquanto isso, lá foraRealizada sobre painel telado com carvão e cera de abelhaEmoldurada em madeira preta", "label": "carvão e cera de abelha sobre painel telado"}
    {"text": "2022 Óleo sobre tela alo eu sou legal", "label": "óleo sobre tela"},
    {"text": "Mulheres do Bordel, déc. 1950. Aquarela e nanquim sobre cartão.", "label": "aquarela e nanquim sobre cartão"}
]

def convert_to_iob_format(data):
    iob_data = []
    for entry in data:
        text = entry['text']
        label = entry['label']
        
        tokens = text.split()
        label_tokens = label.split()
        
        iob_labels = ['O'] * len(tokens)
        
        # Find the start and end positions of the label in the text
        start_pos = text.find(label_tokens[0])
        end_pos = start_pos + len(label_tokens[0])
        
        # Mark the tokens within the label as 'B-LABEL' and 'I-LABEL'
        iob_labels[start_pos:end_pos] = ['B-LABEL'] + ['I-LABEL'] * (len(label_tokens) - 1)
        
        # Create a new dictionary with IOB-formatted labels
        iob_entry = {
            'tokens': tokens,
            'labels': iob_labels
        }
        
        iob_data.append(iob_entry)
    
    return iob_data

iob_data = convert_to_iob_format(data)
print(iob_data)


In [4]:
# Define the labeled dataset and preprocessing functions
# ...

# Create a custom dataset for PyTorch DataLoader
class ArtworkDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, padding='max_length', truncation=True, return_tensors='pt')
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long),
        }

# Define the BERT-based NER model
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=num_labels)

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Define your DataLoader with the custom dataset
train_dataset = ArtworkDataset(train_texts, train_labels, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        
        optimizer.zero_grad()
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluate the model
# ...

# Test the model
# ...



Materials (text1): []
Mediums (text1): []
Materials (text2): []
Mediums (text2): []
